In [2]:
import pandas as pd
import numpy as np

#-----------------------------------------------#

import time
import os
from dotenv import load_dotenv

#-----------------------------------------------#

from spotipy.oauth2 import SpotifyOAuth
import spotipy
import spotipy.util as util

#-----------------------------------------------#

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel

In [3]:
load_dotenv()

True

In [4]:
cid = 'f422d920d33049c09cfe96aa62ea9855'
secret = 'a2daa84d377d4937b00e6b9429c5f20f'
redirect_uri = 'http://localhost:8080'
username = '31yxuvapn3lfzdhax4es457pyutq'

scope = 'user-top-read playlist-modify-public playlist-modify-private'
auth_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope, username=username)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
top20_raw = sp.current_user_top_tracks(time_range='short_term', limit=20)

In [6]:
for i, item in enumerate(top20_raw['items']):
    print(i+1, item['name'], '//', item['artists'][0]['name'])

1 Schatzi bitte - Dream DJ Team Mix // Bitschu Batschu
2 Another Love // Tom Odell
3 Schmetterling // Baby B3ns
4 Cherries // Layfullstop
5 No Hands (feat. Roscoe Dash & Wale) // Waka Flocka Flame
6 I Miss You // blink-182
7 The Rapture Pt.III // &ME
8 Haifischlaserknarre // Mausio
9 Good Guy // Frank Ocean
10 Put Your Records On // Corinne Bailey Rae
11 Palestine // Ali Bumaye
12 Mbappe // Eladio Carrion
13 Krustenf!cker // Mausio
14 Wir haben Spass und ihr nicht // Anna Ullrich
15 TABU. // Yung Yury
16 Frei sein // DREAM DJ TEAM
17 Mädchen auf dem Pferd // Luca-Dante Spadafora
18 Love Yuh Bad // Popcaan
19 Jorja Interlude // Drake
20 Bounce // Joose The Conqueror


In [6]:
tracks = top20_raw['items']
track_ids = []
track_names = []
features = []

for track in tracks:
    track_id = track['id']
    track_name = track['name']
    audio_features = sp.audio_features(track_id)
    
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])

In [7]:
top20_df = pd.DataFrame(features,index = track_names)


In [8]:
top20 = top20_df[['id', 'danceability',	'energy',	'key',	'loudness',	'mode',	'speechiness',	'acousticness',	'instrumentalness',	'liveness',	'valence',	'tempo', 'duration_ms']]


In [9]:
top20.head(5)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
Si No Estás,2HafqoJbgXdtjwCOvNEF14,0.537,0.421,5,-8.720,1,0.0285,0.8270,0.000000,0.1380,0.524,98.224,184061
QLONA,5RqSsdzTNPX1uzkmlHCFvK,0.841,0.738,7,-7.455,0,0.3070,0.5200,0.000000,0.0892,0.484,169.918,172798
Fire On Fire,7t3Xdbufg7q2onVsR8RBdY,0.584,0.408,1,-7.356,0,0.0461,0.4760,0.000000,0.1800,0.333,115.129,246735
MI EX TENÍA RAZÓN,54zcJnb3tp9c5OVKREZ1Is,0.634,0.725,5,-4.945,0,0.0587,0.0776,0.000000,0.2110,0.795,159.845,154375
Tamagotchi,5bfep56ypsHGIR4i5KPKJN,0.870,0.664,1,-8.716,1,0.0744,0.2330,0.000705,0.1410,0.840,98.013,151000


In [10]:
top20.shape

(20, 13)

In [11]:
ids_artists = []
print('|| Artists in my top 20: ||')
print('===========================')
for item in top20_raw['items']:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id)

|| Artists in my top 20: ||
0jbo7KFNMiIkfBR6ih0yhm: iñigo quintero
790FomKkXshlbRYZFtlgla: KAROL G
2wY79sveU1sp5g7SokKOiI: Sam Smith
790FomKkXshlbRYZFtlgla: KAROL G
3psizJPIbIEEctInvdWSZk: GUNTTER
7iK8PXO48WeuP03g8YR51W: Myke Towers
0jbo7KFNMiIkfBR6ih0yhm: iñigo quintero
45dkTj5sMRSjrmBSBeiHym: Tate McRae
790FomKkXshlbRYZFtlgla: KAROL G
22wbnEMDvgVIAGdFeek6ET: Benson Boone
4q3ewBCX7sLwd24euuV69X: Bad Bunny
1i8SpTcr7yvPOmcqrbnVXY: Ozuna
1r4hJ1h58CWwUQe3MxPuau: Maluma
52iwsT98xCoGgiGntTiR7K: Quevedo
52iwsT98xCoGgiGntTiR7K: Quevedo
4bw2Am3p9ji3mYsXNXtQcd: Piso 21
7ltDVBr6mKbRvohxheJ9h1: ROSALÍA
0jbo7KFNMiIkfBR6ih0yhm: iñigo quintero
790FomKkXshlbRYZFtlgla: KAROL G
2R21vXR83lH98kGeO99Y66: Anuel AA


In [12]:
ids_artists = list(set(ids_artists))
print(f'Number of artists (without repetitions): {len(ids_artists)}')

Number of artists (without repetitions): 14


In [13]:
print('Artistas similares:')
print('=====================')
ids_similar_artists = []
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    for item in artists:
        artist_id = item['id']
        artist_name = item['name']
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id)

Artistas similares:
7pwufEBGfggjoI8twqlsmQ: Nessa Barrett
2kRfqPViCqYdSGhYSM9R0Q: Madison Beer
79AyR6ATpj2LTPxfb6FX50: Lauren Spencer Smith
5IH6FPUwQTxPSXurCrcIov: Alec Benjamin
74KM79TiuVKeVCqs8QtB0B: Sabrina Carpenter
1QRj3hoop9Mv5VvHQkwPEp: Olivia O'Brien
4tuJ0bMpJh08umKkEXKUI5: Gracie Abrams
4xnihxcoXWK3UqryOSnbw5: Sasha Alex Sloan
3GxKJzJK4LpsYGXQrw77wz: Mimi Webb
2W8yFh0Ga6Yf3jiayVxwkE: Dove Cameron
2WzaAvm2bBCf4pEhyuDgCY: Ruth B.
4VdV2qRAYBLINR6uU72V1J: Joshua Bassett
7Lk9V7E1u5gqSHmtcKlOqH: Sara Kays
2S054G7qnCK45KY0XzpX30: Dylan Conrique
0ZED1XzwlLHW4ZaG4lOT6m: Julia Michaels
3QSQFmccmX81fWCUSPTS7y: Dean Lewis
2yLzlEFtIS0Q9UkyBZdQA7: Jessie Murph
22wbnEMDvgVIAGdFeek6ET: Benson Boone
4Uc8Dsxct0oMqx0P6i60ea: Conan Gray
7okSU80WTrn4LXlyXYbX3P: Clinton Kane
6w9ToX5slZ4uIdmD17hJ3c: Bryant Myers
7iK8PXO48WeuP03g8YR51W: Myke Towers
3RtNN1VnooWEn3KQk03DUL: Noriel
4SsVbpTthjScTS7U2hmr1X: Arcángel
12vb80Km0Ew53ABfJOepVz: Ñengo Flow
5XJDexmWFLWOkjOEjOVX3e: Eladio Carrion
6nVcHLIgY5pE2YCl

In [14]:
ids_artists.extend(ids_similar_artists)

In [15]:
ids_artists = list(set(ids_artists))
print(f'Number of artists (without repetitions): {len(ids_artists)}')

Number of artists (without repetitions): 187


In [16]:
new_releases = sp.new_releases(limit=20)['albums']

In [17]:
new_releases

{'href': 'https://api.spotify.com/v1/browse/new-releases?country=ES&offset=0&limit=20',
 'items': [{'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Mw40k757jZuiL0NIJpdO5'},
     'href': 'https://api.spotify.com/v1/artists/1Mw40k757jZuiL0NIJpdO5',
     'id': '1Mw40k757jZuiL0NIJpdO5',
     'name': 'GULEED',
     'type': 'artist',
     'uri': 'spotify:artist:1Mw40k757jZuiL0NIJpdO5'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4az97MtWmBQ5Db3GfDh9j9'},
     'href': 'https://api.spotify.com/v1/artists/4az97MtWmBQ5Db3GfDh9j9',
     'id': '4az97MtWmBQ5Db3GfDh9j9',
     'name': 'Morad',
     'type': 'artist',
     'uri': 'spotify:artist:4az97MtWmBQ5Db3GfDh9j9'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'H

In [18]:
print('')
print('Artists with new releases')
print('=====================')
for item in new_releases['items']:
    artist_id = item['artists'][0]['id']   #elegir solo el primer artista
    artist_name = item['artists'][0]['name']
    album_name = item['name']   # Nombre del album, puramente informativo en python
    release_date = item['release_date'] # Fecha de lanzamiento, puramente informativo en python
    print(f'{artist_id}: {artist_name} - // {album_name}, {release_date}')
    ids_artists.append(artist_id)


Artists with new releases
1Mw40k757jZuiL0NIJpdO5: GULEED - // Cuando Menos Lo Espera, 2023-07-07
6k8mwkKJKKjBILo7ypBspl: Ana Mena - // bellodrama, 2023-03-24
7iK8PXO48WeuP03g8YR51W: Myke Towers - // LA VIDA ES UNA, 2023-03-23
5XJDexmWFLWOkjOEjOVX3e: Eladio Carrion - // 3MEN2 KBRN, 2023-03-17
790FomKkXshlbRYZFtlgla: KAROL G - // MAÑANA SERÁ BONITO, 2023-02-24
2auC28zjQyVTsiZKNgPRGs: RM - // Indigo, 2022-12-02
6KImCVD70vtIoJWnq6nGn3: Harry Styles - // Harry's House, 2022-05-20
4q3ewBCX7sLwd24euuV69X: Bad Bunny - // Un Verano Sin Ti, 2022-05-06
2R21vXR83lH98kGeO99Y66: Anuel AA - // Las Leyendas Nunca Mueren, 2021-11-26
4dpARuHxo51G3z768sgnrY: Adele - // 30, 2021-11-19
6eUKZXaKkcviH0Ku9w2n3V: Ed Sheeran - // =, 2021-10-29
4gzpq5DPGxSnKTe4SA8HAU: Coldplay - // Music Of The Spheres, 2021-10-15
53KwLdlmrlCelAZMaLVZqU: James Blake - // Friends That Break Your Heart, 2021-10-08
4MzJMcHQBl9SIYSjwWn8QW: Spiritbox - // Eternal Blue, 2021-09-17
1vyhD5VmyZ7KMfW5gqLgo5: J Balvin - // JOSE, 2021-09-1

In [19]:
ids_artists = list(set(ids_artists))
print(f'Number of artists (without repetitions): {len(ids_artists)}')

Number of artists (without repetitions): 199


In [20]:
id_albums = []
nartists = len(ids_artists)
for i, id_artist in enumerate(ids_artists):
    print(f'Processing artist {i+1} of {nartists}...')
    albums = sp.artist_albums(id_artist, limit=1)  # to avoid having a huge list
    for album in albums['items']:
        id_albums.append(album['id'])
    
    time.sleep(2)  # Adds 3-seconds delay

print('Done!')


Processing artist 1 of 199...
Processing artist 2 of 199...
Processing artist 3 of 199...
Processing artist 4 of 199...
Processing artist 5 of 199...
Processing artist 6 of 199...
Processing artist 7 of 199...
Processing artist 8 of 199...
Processing artist 9 of 199...
Processing artist 10 of 199...
Processing artist 11 of 199...
Processing artist 12 of 199...
Processing artist 13 of 199...
Processing artist 14 of 199...
Processing artist 15 of 199...
Processing artist 16 of 199...
Processing artist 17 of 199...
Processing artist 18 of 199...
Processing artist 19 of 199...
Processing artist 20 of 199...
Processing artist 21 of 199...
Processing artist 22 of 199...
Processing artist 23 of 199...
Processing artist 24 of 199...
Processing artist 25 of 199...
Processing artist 26 of 199...
Processing artist 27 of 199...
Processing artist 28 of 199...
Processing artist 29 of 199...
Processing artist 30 of 199...
Processing artist 31 of 199...
Processing artist 32 of 199...
Processing artist

In [21]:
id_albums

['0pWzIntouGCOZzQsSW6xYc',
 '7JX96Ff9zr74IvC1qGyBvM',
 '65LmFuLBa1yBX5yfuuOMh2',
 '465QWT41lSOGkWGsOeFPO5',
 '4aAHlX9XKBRfdqUp4JVvr0',
 '5bXx1ncSvWPtNAuecQgsHf',
 '3Fk8yQvPlCHgwR2pNhEIRA',
 '5EmcDn0tz8o9C2OEwtKR0X',
 '14d0QzazggZIpjO2dtkRWo',
 '27fFLYwlXF1x8n1ixpFbFH',
 '7IGxggYiFgj0mtP7BTQFbO',
 '7vXNMveRwef98O1M6I3Lrb',
 '7ntr88BWCrwZGySZQlHgBM',
 '7fkUNsg2oiqWaBtkO4ZasS',
 '17cNWP1FoUxSTwr3ki8MnX',
 '4AHHM76jyqUiC1HRd37ZQQ',
 '1KgKemleH5GeTmcUA7U9ka',
 '3XlG8WaWzdjhDXuaRmOlaC',
 '7djdwVJvXXMDXTLbBPzRzh',
 '5WgC0rNbr8aC5vEqL9U8Zo',
 '6mBNXZGN2V1JmcUhjlxU3q',
 '2pfBdekMfDRVf1MfDnwTSv',
 '11UJfH39O0VqLiQGLW0D0e',
 '2dIHjjKPHRbon4HCfCvyuE',
 '5bLE2kmkZWlzAYeb6To6LA',
 '4OXoBlapQygTdzAifJm8BL',
 '0Gah9MaqGz3PcFpGAiJeWb',
 '00dOLQ7BxGTZIUKZcX8AGY',
 '5reEg9or7xqpXy6XLVIoTE',
 '79JpJRorp0NHWcAUUXpTJP',
 '7nz20mGwRFgfR1K67RNh9q',
 '4coTyK7vESCTGwrtbHPOKn',
 '3UxgQmk5mb2G4zCFM4ScHH',
 '5gSUafhGxfjgna6Wr0fiVc',
 '4BvD9caT3ZohqNQFcR8dVD',
 '2a94BZemddihoxTNN0Oh7a',
 '76pGPt58PuyQrugKnLD0eO',
 

In [23]:
id_tracks = []
nalbums = len(id_albums)
for i, id_album in enumerate(id_albums):
    print(f'Processing album {i+1} of {nalbums}...')
    album_tracks = sp.album_tracks(id_album, limit=2)
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
    
    time.sleep(2)  # Adds 3-seconds delay

print(f'Done! Total number of pre-candidate tracks: {len(id_tracks)}')


Processing album 1 of 199...
Processing album 2 of 199...
Processing album 3 of 199...
Processing album 4 of 199...
Processing album 5 of 199...
Processing album 6 of 199...
Processing album 7 of 199...
Processing album 8 of 199...
Processing album 9 of 199...
Processing album 10 of 199...
Processing album 11 of 199...
Processing album 12 of 199...
Processing album 13 of 199...
Processing album 14 of 199...
Processing album 15 of 199...
Processing album 16 of 199...
Processing album 17 of 199...
Processing album 18 of 199...
Processing album 19 of 199...
Processing album 20 of 199...
Processing album 21 of 199...
Processing album 22 of 199...
Processing album 23 of 199...
Processing album 24 of 199...
Processing album 25 of 199...
Processing album 26 of 199...
Processing album 27 of 199...
Processing album 28 of 199...
Processing album 29 of 199...
Processing album 30 of 199...
Processing album 31 of 199...
Processing album 32 of 199...
Processing album 33 of 199...
Processing album 34

In [24]:
id_tracks

['04v1IMQMdemyNhfH5G2uyp',
 '6r0TaoZh1Oc1KkuL4rC1kB',
 '0fx9L2QDi387TQQOVnm91O',
 '5BHMS8LBI2TYTdObzteZj0',
 '1QBoyx9EOFbnFQJGCyEzwz',
 '3yh46UlxlTAFpTSaQI3XDm',
 '2t4SJrRZrGOEQlNDabywTw',
 '0DNzQupfod8PPlGDc73kvK',
 '0mXjm33kyUTMkgJNOKvGOO',
 '4zzP8gZCqTlbt1gzDeQMXd',
 '4r4yF6BGAm4oy4MnOYjSjI',
 '40dfo65kBJa1UgzkD652Gl',
 '6toBEs42KExYtyPzlEf4eu',
 '0XjX9s0XWOKZc8PLuW02wJ',
 '0Ckmi4rrdUWnlg4T9aIGCS',
 '6Jt18cyNzFXYYUUBllm7rZ',
 '1AozLLbKi1kIKMUbY3Xk7o',
 '5KA3dYKykvZW5KBEpSTmOL',
 '6PSyxMi8vTq5m25Yh6Oc6J',
 '3AsbVi6XAEGXoKOTfhzevi',
 '3HrfjIm4usDUk3r4IpvLeX',
 '5Odeka5MkBqFeABthrtagH',
 '6hWwvq0NVF8yedQrHAuEYr',
 '4lbN59oAXTZuKkq16ACHbE',
 '6v3oRcV32Jj1zzFjBRspfq',
 '5WYzgXAzOP814ninL9deX2',
 '4R93vEvbVvzWjq9C6ZNHUR',
 '6iU3wBJNNSHbtac32TATEh',
 '1CcCntDuVJpjqkJhYGhuMw',
 '0A6rhK1bckxbfdOm1Omjq2',
 '2YZDh5CecWx0qlFiTvqVQ9',
 '4bswq6hRPqJ56vIpCFBbRZ',
 '2GBfmqKw2YWLu5wh9Ok321',
 '2Xkxoluz9GQnQYAjxXImGG',
 '6ys2DESjeA2Q22ntjHscok',
 '0VJDKFCfJJyA8CgMd2Cygj',
 '7x8efoIjgyyQGogsHltYFf',
 

In [25]:
track_names = []
features = []
ntracks = len(id_tracks)
for i, track_id in enumerate(id_tracks):
    print(f'Processing track {i+1} of {ntracks}...')
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    
    # Do not include tracks without "features"
    if audio_features[0] != None:
        track_names.append(track_name)
        features.append(audio_features[0])

    time.sleep(2)  # Adding some seconds delay

print('Done!')

candidates_df = pd.DataFrame(features, index = track_names)

Processing track 1 of 363...
Processing track 2 of 363...
Processing track 3 of 363...
Processing track 4 of 363...
Processing track 5 of 363...
Processing track 6 of 363...
Processing track 7 of 363...
Processing track 8 of 363...
Processing track 9 of 363...
Processing track 10 of 363...
Processing track 11 of 363...
Processing track 12 of 363...
Processing track 13 of 363...
Processing track 14 of 363...
Processing track 15 of 363...
Processing track 16 of 363...
Processing track 17 of 363...
Processing track 18 of 363...
Processing track 19 of 363...
Processing track 20 of 363...
Processing track 21 of 363...
Processing track 22 of 363...
Processing track 23 of 363...
Processing track 24 of 363...
Processing track 25 of 363...
Processing track 26 of 363...
Processing track 27 of 363...
Processing track 28 of 363...
Processing track 29 of 363...
Processing track 30 of 363...
Processing track 31 of 363...
Processing track 32 of 363...
Processing track 33 of 363...
Processing track 34

implementing a simple caching mechanism that could help reduce the number of API calls
    
    track_cache = {}


    track_names = []
    features = []
    ntracks = len(id_tracks)

    for i, track_id in enumerate(id_tracks):
        print(f'Processing track {i+1} of {ntracks}...')

        # Check if the track's details are in the cache
        if track_id in track_cache:
            track_info = track_cache[track_id]
        else:
            track_info = sp.track(track_id)
            track_cache[track_id] = track_info  # Cache the data for future use
            time.sleep(3)  # Adding some seconds delay to avoid rate limit

        track_name = track_info['name']
        audio_features = sp.audio_features(track_id)

        # Check if the audio features are in the cache
        if track_id in track_cache and 'features' in track_cache[track_id]:
            features_info = track_cache[track_id]['features']
        else:
            features_info = audio_features[0] if audio_features and audio_features[0] is not None else None
            if features_info:
                track_cache[track_id]['features'] = features_info  # Cache the features

        if features_info:
            track_names.append(track_name)
            features.append(features_info)

    print('Done!')

    candidates_df = pd.DataFrame(features, index=track_names)


In [26]:
candidates_df.head(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Ya No,0.759,0.339,10,-7.314,0,0.0274,0.9400,0.003600,0.1100,0.470,98.047,audio_features,04v1IMQMdemyNhfH5G2uyp,spotify:track:04v1IMQMdemyNhfH5G2uyp,https://api.spotify.com/v1/tracks/04v1IMQMdemy...,https://api.spotify.com/v1/audio-analysis/04v1...,171851,3
DISPARA ***,0.810,0.657,1,-2.978,1,0.1550,0.4800,0.000000,0.1080,0.698,89.972,audio_features,6r0TaoZh1Oc1KkuL4rC1kB,spotify:track:6r0TaoZh1Oc1KkuL4rC1kB,https://api.spotify.com/v1/tracks/6r0TaoZh1Oc1...,https://api.spotify.com/v1/audio-analysis/6r0T...,143936,4
Everything,0.752,0.904,8,-2.846,1,0.0698,0.0206,0.000000,0.0723,0.527,98.034,audio_features,0fx9L2QDi387TQQOVnm91O,spotify:track:0fx9L2QDi387TQQOVnm91O,https://api.spotify.com/v1/tracks/0fx9L2QDi387...,https://api.spotify.com/v1/audio-analysis/0fx9...,183787,4
La Llamada,0.805,0.694,4,-6.634,0,0.0480,0.2610,0.000638,0.2180,0.732,95.058,audio_features,5BHMS8LBI2TYTdObzteZj0,spotify:track:5BHMS8LBI2TYTdObzteZj0,https://api.spotify.com/v1/tracks/5BHMS8LBI2TY...,https://api.spotify.com/v1/audio-analysis/5BHM...,199587,4
Love Me For Another Day,0.536,0.423,6,-9.158,1,0.0314,0.6950,0.000185,0.1170,0.228,134.027,audio_features,1QBoyx9EOFbnFQJGCyEzwz,spotify:track:1QBoyx9EOFbnFQJGCyEzwz,https://api.spotify.com/v1/tracks/1QBoyx9EOFbn...,https://api.spotify.com/v1/audio-analysis/1QBo...,183000,3


In [27]:
candidates = candidates_df[['id', 'danceability',	'energy',	'key',	'loudness',	'mode',	'speechiness',	'acousticness',	'instrumentalness',	'liveness',	'valence',	'tempo', 'duration_ms']]


In [28]:
candidates.to_csv('./spotify_data_cache/mi_base_de_datos.csv', index=False)

In [29]:
candidates = pd.read_csv('./datos/mi_base_de_datos.csv')

In [30]:
top20_mtx = top20.iloc[:,1:].values
candidatos_mtx = candidates.iloc[:,1:].values

In [31]:
scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
can_scaled = scaler.fit_transform(candidatos_mtx)

In [32]:
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
can_norm = np.sqrt((can_scaled*can_scaled).sum(axis=1))

In [33]:
nt20 = t20_scaled.shape[0]
ncan = can_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
can = can_scaled/can_norm.reshape(ncan,1)

In [34]:
cos_sim = linear_kernel(t20,can)
cos_sim.shape

(20, 995)

In [35]:
print(cos_sim[6,270])
print(cos_sim[3,24])

0.08176107395661335
-0.31220173087663866


In [36]:
def obtener_candidatos(pos, cos_sim, ncands, umbral = 0.8):
    
    # Obtener todas las pistas candidatas por encima de umbral
    
    idx = np.where(cos_sim[pos,:]>=umbral)[0] 
    
    # Y organizarlas de forma descendente (por similitudes de mayor a menor)
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] porque por defecto argsort organiza de manera ascendente

    # Si hay más de "ncands", retornar máximo "ncands"
    if len(idx) >= ncands:
        cands = idx[0:ncands]
    else:
        cands = idx
  
    return cands

In [37]:
for i in range(5):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i}==> pistas filtradas: {cands}, similitudes: {cos_sim[i,cands]}')

0==> pistas filtradas: [ 42 162 572 621 921], similitudes: [0.93658215 0.92423247 0.92207399 0.91621042 0.9119082 ]
1==> pistas filtradas: [861  87], similitudes: [0.82628737 0.82563557]
2==> pistas filtradas: [], similitudes: []
3==> pistas filtradas: [685 816 281 987 794], similitudes: [0.8515682  0.84035848 0.82829469 0.80594698 0.80426107]
4==> pistas filtradas: [], similitudes: []


In [38]:
ids_t20 = []
ids_playlist = []

for i in range(top20_df.shape[0]):
    print(top20_df.index[i]) 
    ids_t20.append(top20_df['id'][i])
    
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    
    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        for j in cands:
            id_cand = candidates['id'][j]
            ids_playlist.append(id_cand)
            
            print(f'   {candidates.index[j]}')

Si No Estás
   42
   162
   572
   621
   921
QLONA
   861
   87
Fire On Fire
     ***No se encontraron pistas relacionadas***
MI EX TENÍA RAZÓN
   685
   816
   281
   987
   794
Tamagotchi
     ***No se encontraron pistas relacionadas***
LALA
   80
   267
   358
   838
   359
Sin Tiempo Para Bailar
   70
   754
greedy
     ***No se encontraron pistas relacionadas***
AMARGURA
   395
   983
   384
   168
In The Stars
     ***No se encontraron pistas relacionadas***
PERRO NEGRO
   551
Hey Mor
   199
Según Quién
   293
   261
   290
   958
   490
Columbia
   428
WANDA
   816
Déjala Que Vuelva (feat. Manuel Turizo)
   498
   841
   376
   169
   48
BESO
     ***No se encontraron pistas relacionadas***
Sobredosis
   566
   670
   846
   893
   617
QLONA
   861
Hasta Que Dios Diga
     ***No se encontraron pistas relacionadas***


C:\Users\Felix\AppData\Local\Temp\ipykernel_18816\1504939720.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ids_t20.append(top20_df['id'][i])


In [39]:
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]
ids_playlist_dep = list(set(ids_playlist_dep))

In [40]:
print(len(ids_playlist_dep))

41


In [44]:
# Now try creating the playlist
pl = sp.user_playlist_create(user=username, 
                             name='Spotipy Recommender Playlist', 
                             description="Playlist created with the recommendation system")

sp.playlist_add_items(pl['id'],ids_playlist_dep)

{'snapshot_id': 'Myw4MTEwMTVjMWI1NWNiOWEyYjJmMjE4YzVmYjVhMTA5N2FjZWY2MmYx'}